In [2]:
import pandas as pd
import numpy as np
import os
import warnings
import random
from collections import Counter
import subprocess
pd.options.mode.chained_assignment = None

In [5]:
# get the intergenic regions created by bedtools' subtractBed function. "hg38_whole_chr.bed" simply lists the start/end of each chromosome, and the annotation file was retrieved from UCSC
os.system('subtractBed -a hg38_whole_chr.bed -b hg38.ncbiRefSeq.bed.gz > hg38.ncbiRefSeq.intergenic.bed')
os.system('gzip hg38.ncbiRefSeq.intergenic.bed')

# load NCBI chr12 annotation
ncbi = pd.read_csv('hg38.ncbiRefSeq.intergenic.bed.gz', sep='\t', header=None, names=['chr', 'start', 'end'])
ncbi = ncbi[ncbi['chr'] == 'chr12']
ncbi['start'] = ncbi['start'].astype(int)
ncbi['end'] = ncbi['end'].astype(int)

# select start and end for each chromosome
whole_chr_coor = pd.read_csv('hg38_whole_chr.bed', sep='\t', header=None)
whole_chr_coor.rename(columns={0: 'chr', 1: 'chr_start', 2: 'chr_end'}, inplace=True)


In [6]:
# RefSeq aligned by NCBI

# add the min/max chr values
ncbi_whole = pd.merge(ncbi, whole_chr_coor, how='left', on='chr')

# get the min/max start/end for use in restricting regions for adding/substracting 10kbp
min_start = set(ncbi_whole['start'])
min_start_value = sorted(min_start)[1] # 32015
max_start_value = sorted(min_start)[-1] # 133238438

max_end = set(ncbi_whole['end'])
max_end_value = sorted(max_end)[-2] # 133275309
min_end_value = sorted(max_end)[0] # 12309


In [7]:
# remove any regions that are not 10kbp from each end that are at least 141bp in length
ncbi_whole['seq_len'] = ncbi_whole['end'] - ncbi_whole['start']
ncbi_ig = ncbi_whole[ncbi_whole['seq_len'] >= 20141]


In [8]:
# update the coordinates

# adds/subtracts 10kbp from each end of the intergenic regions except from the start/end of the chromosome
ncbi_ig['start'] = ncbi_ig['start'].apply(lambda x: x + 10000 if (max_end_value + 10000) >= x >= (min_start_value - 10000) else x)
ncbi_ig['end'] = ncbi_ig['end'].apply(lambda x: x - 10000 if (max_start_value - 10000) >= x >= (min_end_value - 10000) else x)

# get the intergenic region length
ncbi_ig['seq_len'] = ncbi_ig['end'] - ncbi_ig['start']
ncbi_ig.drop(['chr_start', 'chr_end'], axis=1, inplace=True)


In [10]:
# get centromere coordinates. Centromere coordinates from UCSC
centromeres = pd.read_csv('centromeres.txt.gz', header=None, usecols=[1, 2, 3], sep='\t')
centromeres_12 = centromeres[centromeres[1] == 'chr12']

cent_regions = list(zip(centromeres_12[2], centromeres_12[3]))

# centremere for chr 12 is [(34769407, 34816611), (34835295, 37185252)], but I'm just going to merge them as a single region
cent_range = (34769407, 37185252)

def remove_cent(start, end, cent_range):
    '''
    function to remove the centromeric regions from chr12
    '''
    if start < cent_range[0] < end and start < cent_range[1] < end:
        return [(start, cent_range[0] - 1), (cent_range[1] + 1, end)]
    elif start < cent_range[0] < end:
        return [(start, cent_range[0] - 1)]
    elif start < cent_range[1] < end:
        return [(cent_range[1] + 1, end)]
    else:
        return [(start, end)]

# create "new_ranges" column that fixes any intergenic regions overlap with centromere
ncbi_ig['new_ranges'] = ncbi_ig.apply(lambda row: remove_cent(row['start'], row['end'], cent_range), axis=1)

# explode the column, as the affected intergenic region completely encompasses the centromere
ncbi_ig = ncbi_ig.explode('new_ranges').reset_index(drop=True)

# split new_ranges as start and end columns
ncbi_ig[['start', 'end']] = pd.DataFrame(ncbi_ig['new_ranges'].tolist(), index=ncbi_ig.index)

# drop new_ranges column
ncbi_ig.drop(columns=['new_ranges'], inplace=True)

# update the seq_len
ncbi_ig['seq_len'] = ncbi_ig['end'] - ncbi_ig['start']


In [11]:
# get random intergenic regions. To get 1000 random regions, we randomly sampled each region 1000 times, then selected non-overlapping 1000 values from that list

# empty lists to store randomly sampled ranges and corresponding "chr" values
rand_ig = []

# set seed for np. Will also use this value for later random processes
np.random.seed(63)

# set number of ranges & range size
num_ranges = 1000
range_size = 141

# iterate over each row in the dataframe
for index, row in ncbi_ig.iterrows():
    # get the chr, start, and end for the current row
    start_value = row['start']
    end_value = row['end']
    chr_value = row['chr']

    # randomly sample 1000 start coordinates. Need at least 141 bp in length, so the end_value - range_size + 1
    random_samples = np.random.randint(start_value, end_value - range_size + 1, num_ranges)

    # add the sampled ranges and corresponding "chr" values to the lists
    for rand_seq_start in random_samples:
        rand_seq_end = rand_seq_start + range_size

        rand_ig.append({
            'chr': chr_value,
            'intergenic_region_start': start_value,
            'intergenic_region_end': end_value,
            'rand_seq_start': rand_seq_start,
            'rand_seq_end': rand_seq_end
        })
ncbi_ig_rand = pd.DataFrame(rand_ig)

In [12]:
# get random non-overlapping regions

# extract ranges
ranges = list(zip(ncbi_ig_rand['chr'], ncbi_ig_rand['rand_seq_start'], ncbi_ig_rand['rand_seq_end']))

random.seed(63)

# shuffle the list of tuples for randomness
random.shuffle(ranges)

# get 1000 non-overlapping regions
random_ranges = []
for i in ranges:
    if not any(start <= i[2] and end >= i[1] for _, start, end in random_ranges):
        random_ranges.append(i)
        if len(random_ranges) == 1000:
            break

# convert the selected regions as dataframe, then merge with the original to check for the intergenic regions
rand_ig_reg = pd.DataFrame(random_ranges, columns=['chr', 'rand_seq_start', 'rand_seq_end'])
rand_ig_reg_merged = pd.merge(rand_ig_reg, ncbi_ig_rand, on=['chr', 'rand_seq_start', 'rand_seq_end'], how='left').drop_duplicates()

# rearrange the dataframe, then sort by start values for viewing pleasure
rand_ig_reg_merged = rand_ig_reg_merged[['chr', 'intergenic_region_start', 'intergenic_region_end', 'rand_seq_start', 'rand_seq_end']]
rand_ig_reg_merged = rand_ig_reg_merged.sort_values('rand_seq_start')


In [13]:
#  check for overlap
def check_overlap(row, df):
    '''
    function to check for overlap between random intergenic regions
    '''
    overlaps = []
    for index, other_row in df.iterrows():
        if (row['rand_seq_start'] < other_row['rand_seq_end'] and row['rand_seq_end'] > other_row['rand_seq_start']):
            overlaps.append(index)
    return overlaps

overlapping_ranges = []
for index, row in rand_ig_reg_merged.iterrows():
    overlaps = check_overlap(row, rand_ig_reg_merged.drop(index))
    if overlaps:
        overlapping_ranges.append((index, overlaps))

for index, overlaps in overlapping_ranges:
    print(f'{index} overlaps with {overlaps}')

In [14]:
rand_ig_reg_merged.to_csv('results/hg38_ncbiRefSeq_chr12_intergenic_nonoverlap_randseq_no_centremere_updated.txt', index=False, sep='\t')